In [9]:
#data cleanup
import pandas as pd
df_raw = pd.read_csv("Data.csv")

#get basic info. tells us there are no null values
df_raw.info()

#take a sample so that we can see what columns are categories
df_raw.sample(n=5)

#change dtype from object to category
digital_habits = pd.read_csv("Data.csv", dtype = {'gender':'category', 'region':'category', 'income_level':'category', 'daily_role':'category'})

#double check that there truly are no missing values
print(digital_habits.isna().sum())
#yep, no missing values. moving to a new cell.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        3500 non-null   int64  
 1   age                       3500 non-null   int64  
 2   gender                    3500 non-null   object 
 3   region                    3500 non-null   object 
 4   income_level              3500 non-null   object 
 5   education_level           3500 non-null   object 
 6   daily_role                3500 non-null   object 
 7   device_hours_per_day      3500 non-null   float64
 8   phone_unlocks             3500 non-null   int64  
 9   notifications_per_day     3500 non-null   int64  
 10  social_media_mins         3500 non-null   int64  
 11  study_mins                3500 non-null   int64  
 12  physical_activity_days    3500 non-null   float64
 13  sleep_hours               3500 non-null   float64
 14  sleep_qu

In [10]:
#look for any outliers in the data


,id,age,gender,region,income_level,education_level,daily_role,device_hours_per_day,phone_unlocks,notifications_per_day,...,sleep_quality,anxiety_score,depression_score,stress_level,happiness_score,focus_score,high_risk_flag,device_type,productivity_score,digital_dependence_score
1848,1849,20,Male,Asia,Lower-Mid,Bachelor,Student,5.19,111,323,...,2.035652,6.000000,8.0,1.000000,8.5,57.0000,0,Android,75.0,27.2
1603,1604,33,Female,Africa,Upper-Mid,Bachelor,Part-time/Shift,6.64,116,331,...,3.107035,6.390366,8.0,10.000000,5.3,51.0000,0,Tablet,57.0,33.0
3259,3260,36,Female,Asia,Upper-Mid,PhD,Caregiver/Home,7.73,102,115,...,3.896571,6.873503,7.0,1.180825,9.4,58.0000,0,Android,79.0,28.5
1653,1654,32,Female,Africa,High,Bachelor,Part-time/Shift,3.24,51,338,...,3.287453,8.594793,0.0,7.748607,3.0,32.0000,0,Tablet,61.0,19.3
1633,1634,22,Female,Africa,Lower-Mid,Bachelor,Student,6.69,130,285,...,2.955383,6.461315,8.0,1.000000,9.8,50.0000,0,Android,75.0,31.6
941,942,13,Female,Europe,Lower-Mid,High School,Student,6.45,122,413,...,4.236283,7.107964,5.0,1.000000,8.0,41.0000,0,iPhone,74.0,32.8
1487,1488,37,Female,Middle East,High,High School,Part-time/Shift,7.22,121,387,...,1.158862,6.111579,14.0,10.000000,1.3,44.0000,0,iPhone,47.0,36.9
2869,2870,31,Female,South America,Low,Bachelor,Full-time Employee,6.38,118,601,...,3.110957,2.804758,10.0,4.886282,9.7,41.5986,0,Android,77.0,38.6
2552,2553,24,Male,North America,Lower-Mid,Bachelor,Unemployed_Looking,7.66,170,505,...,2.059424,4.000000,9.0,4.702245,8.6,27.0000,0,Tablet,84.0,42.8
1612,1613,21,Male,Europe,Lower-Mid,Bachelor,Unemployed_Looking,6.92,196,211,...,1.000000,6.000000,20.0,10.000000,4.2,64.0000,0,Android,74.0,35.8
